In [3]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy


app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'

db = SQLAlchemy(app)

In [4]:
engine = db.create_engine('sqlite:///datacamp.sqlite1', echo = True)
conn = engine.connect()
metadata = db.MetaData()

Student = db.Table('student', metadata,
                    db.Column('Id', db.Integer(), primary_key = True),
                    db.Column('Name', db.String(255), nullable = False),
                    db.Column('Major', db.String(255), default = 'Maths'),
                    db.Column('Pass', db.Boolean(), default = True)
                )
metadata.create_all(engine)

2024-02-28 10:47:49,399 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-28 10:47:49,404 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("student")
2024-02-28 10:47:49,405 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-28 10:47:49,413 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("student")
2024-02-28 10:47:49,414 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-28 10:47:49,416 INFO sqlalchemy.engine.Engine 
CREATE TABLE student (
	"Id" INTEGER NOT NULL, 
	"Name" VARCHAR(255) NOT NULL, 
	"Major" VARCHAR(255), 
	"Pass" BOOLEAN, 
	PRIMARY KEY ("Id")
)


2024-02-28 10:47:49,420 INFO sqlalchemy.engine.Engine [no key 0.00403s] ()
2024-02-28 10:47:49,586 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
output = conn.execute(Student.select()).fetchall()
print(output)

2024-02-28 10:51:56,888 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student
2024-02-28 10:51:56,890 INFO sqlalchemy.engine.Engine [generated in 0.00131s] ()
[]


In [14]:
query = db.insert(Student)
values = [{'Id':11, 'Name':'Ravi1', 'Major':'Science', 'Pass':True},
          {'Id':21, 'Name':'Raj1', 'Major':'Maths', 'Pass':False},
          {'Id':31, 'Name':'Rahul1', 'Major':'Arts', 'Pass':True},
          {'Id':41, 'Name':'Rohit1', 'Major':'Science', 'Pass':False}]

Result = conn.execute(query, values)

2024-02-28 11:00:27,127 INFO sqlalchemy.engine.Engine INSERT INTO student ("Id", "Name", "Major", "Pass") VALUES (?, ?, ?, ?)
2024-02-28 11:00:27,128 INFO sqlalchemy.engine.Engine [cached since 452.3s ago] [(11, 'Ravi1', 'Science', 1), (21, 'Raj1', 'Maths', 0), (31, 'Rahul1', 'Arts', 1), (41, 'Rohit1', 'Science', 0)]


In [15]:
output = conn.execute(Student.select()).fetchall()
print(output)

2024-02-28 11:00:37,648 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student
2024-02-28 11:00:37,649 INFO sqlalchemy.engine.Engine [cached since 520.8s ago] ()
[(1, 'Ravi', 'Science', True), (2, 'Raj', 'Maths', False), (3, 'Rahul', 'Arts', True), (4, 'Rohit', 'Science', False), (11, 'Ravi1', 'Science', True), (21, 'Raj1', 'Maths', False), (31, 'Rahul1', 'Arts', True), (41, 'Rohit1', 'Science', False)]


In [16]:
Result.connection.commit()
Result.close()

2024-02-28 11:01:54,500 INFO sqlalchemy.engine.Engine COMMIT


In [21]:
query = Student.select().where(Student.columns.Major == 'Maths')
output = conn.execute(query).fetchall()
print(output)

2024-02-28 11:05:15,803 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student 
WHERE student."Major" = ?
2024-02-28 11:05:15,804 INFO sqlalchemy.engine.Engine [cached since 35.81s ago] ('Maths',)
[(2, 'Raj', 'Maths', False), (21, 'Raj1', 'Maths', False)]


In [24]:
query = Student.select().where(db.and_(Student.columns.Major == 'Maths', Student.columns.Pass != True))
output = conn.execute(query).fetchall()
print(output)

2024-02-28 11:11:46,791 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student 
WHERE student."Major" = ? AND student."Pass" != 1
2024-02-28 11:11:46,793 INFO sqlalchemy.engine.Engine [generated in 0.00137s] ('Maths',)
[(2, 'Raj', 'Maths', False), (21, 'Raj1', 'Maths', False)]


In [25]:
query = Student.select().where(db.or_(Student.columns.Major == 'Maths', Student.columns.Pass == True))
output = conn.execute(query).fetchall()
print(output)

2024-02-28 11:12:52,265 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student 
WHERE student."Major" = ? OR student."Pass" = 1
2024-02-28 11:12:52,266 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ('Maths',)
[(1, 'Ravi', 'Science', True), (2, 'Raj', 'Maths', False), (3, 'Rahul', 'Arts', True), (11, 'Ravi1', 'Science', True), (21, 'Raj1', 'Maths', False), (31, 'Rahul1', 'Arts', True)]


In [27]:
query = Student.select().where(Student.columns.Major.in_(['Science', 'Maths']))
output = conn.execute(query).fetchall()
print(output)

2024-02-28 11:13:35,409 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student 
WHERE student."Major" IN (?, ?)
2024-02-28 11:13:35,411 INFO sqlalchemy.engine.Engine [generated in 0.00456s] ('Science', 'Maths')
[(1, 'Ravi', 'Science', True), (2, 'Raj', 'Maths', False), (4, 'Rohit', 'Science', False), (11, 'Ravi1', 'Science', True), (21, 'Raj1', 'Maths', False), (41, 'Rohit1', 'Science', False)]


In [32]:
query = Student.select().order_by(db.desc(Student.columns.Name))
output = conn.execute(query).fetchall()
print(output)

2024-02-28 11:14:52,406 INFO sqlalchemy.engine.Engine SELECT student."Id", student."Name", student."Major", student."Pass" 
FROM student ORDER BY student."Name" DESC
2024-02-28 11:14:52,407 INFO sqlalchemy.engine.Engine [generated in 0.00115s] ()
[(41, 'Rohit1', 'Science', False), (4, 'Rohit', 'Science', False), (11, 'Ravi1', 'Science', True), (1, 'Ravi', 'Science', True), (21, 'Raj1', 'Maths', False), (2, 'Raj', 'Maths', False), (31, 'Rahul1', 'Arts', True), (3, 'Rahul', 'Arts', True)]


In [33]:
print(Student.columns.keys())

['Id', 'Name', 'Major', 'Pass']


In [37]:
print(repr(metadata.tables['student']))

Table('student', MetaData(), Column('Id', Integer(), table=<student>, primary_key=True, nullable=False), Column('Name', String(length=255), table=<student>, nullable=False), Column('Major', String(length=255), table=<student>, default=ScalarElementColumnDefault('Maths')), Column('Pass', Boolean(), table=<student>, default=ScalarElementColumnDefault(True)), schema=None)
